Κατασκευή train και val dataset

In [1]:
import pyvww
import torch
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

training_set = pyvww.pytorch.VisualWakeWordsClassification(
    root="./visualwakewords/path-to-COCO-dataset/train2014",
    annFile="./visualwakewords/new-path-to-visualwakewords-dataset/annotations/instances_train.json",
    transform=transform
)

validation_set = pyvww.pytorch.VisualWakeWordsClassification(
    root="./visualwakewords/path-to-COCO-dataset/train2014",
    annFile="./visualwakewords/new-path-to-visualwakewords-dataset/annotations/instances_val.json",
    transform=transform
)



training_loader = torch.utils.data.DataLoader(training_set, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

loading annotations into memory...
Done (t=4.71s)
creating index...
index created!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os
import time


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def train_one_epoch(model, optimizer, loss_fn, epoch):
    train_loss, train_correct, train_total = 0.0, 0, 0

    for batch_idx, (inputs, labels) in enumerate(training_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

        if batch_idx % 25 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}] | Training Batch [{batch_idx+1}/{len(training_loader)}] | Loss: {loss.item():.4f}")

    return train_loss, train_correct, train_total

In [4]:
def train_and_val(model, optimizer, loss_fn, epochs, log_file_path, model_file_path):
    model.to(device)
    best_vacc = 0
    train_accuracies = []
    val_accuracies = []

    print(f"Model is running on: {device}")
    
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}:')

        # Training Phase
        model.train() 
        train_loss, train_correct, train_total = train_one_epoch(model, optimizer, loss_fn, epoch)

        # Calculate training accuracy
        train_acc = train_correct / train_total
        train_accuracies.append(train_acc)

        # Validation Phase
        model.eval() 
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for batch_idx, (vinputs, vlabels) in enumerate(validation_loader):
                vinputs, vlabels = vinputs.to(device), vlabels.to(device)

                voutputs = model(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                val_loss += vloss.item() * vinputs.size(0)

                _, vpreds = torch.max(voutputs, 1)
                val_correct += (vpreds == vlabels).sum().item()
                val_total += vlabels.size(0)
                
                if batch_idx % 25 == 0:  # Print every 10 batches
                    print(f"Epoch [{epoch+1}] | Validation Batch [{batch_idx+1}/{len(training_loader)}] | Loss: {val_loss:.4f}")

        # Calculate validation accuracy
        val_acc = val_correct / val_total
        val_accuracies.append(val_acc)

        print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

        with open(log_file_path, "a") as log_file:
            if os.stat(log_file_path).st_size == 0:
                log_file.write("Epoch\tTrain_Loss\tTrain_Acc\tVal_Loss\tVal_Acc\n")
            
            log_file.write(f"{epoch + 1}\t{train_loss:.4f}\t{train_acc:.4f}\t{val_loss:.4f}\t{val_acc:.4f}\n")

        # Save model if it's the best validation loss so far
        if val_acc > best_vacc:
            best_vacc = val_acc
            torch.save(model.state_dict(), model_file_path)
            print("Saved model with best performance so far.")

    return train_accuracies, val_accuracies

In [4]:
torch.cuda.empty_cache() 

In [7]:
import torch.cuda
import threading

stream1 = torch.cuda.Stream()
stream2 = torch.cuda.Stream()

def train_model1():
    model = models.shufflenet_v2_x0_5(weights=models.ShuffleNet_V2_X0_5_Weights.DEFAULT).cuda()

    loss_function = nn.CrossEntropyLoss()

    #Freeze all layers except the last FC layer
    for param in model.parameters():
        param.requires_grad = False

    model.fc=nn.Linear(1024, 2)
    model.fc.requires_grad = True

    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    start_time = time.time()
    with torch.cuda.stream(stream1):
        train_and_val(model, optimizer, loss_function, 10, "./results/shufflenet_v2_x0_5.txt", "./shufflenet_v2_x0_5.pth")
    end_time = time.time()
    print(f"Training time for Model 1: {end_time - start_time:.2f} seconds")


def train_model2():
    model = models.shufflenet_v2_x1_0(weights=models.ShuffleNet_V2_X1_0_Weights.DEFAULT).cuda()

    loss_function = nn.CrossEntropyLoss()

    #Freeze all layers except the last FC layer
    for param in model.parameters():
        param.requires_grad = False

    model.fc=nn.Linear(1024, 2)
    model.fc.requires_grad = True

    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    start_time = time.time()
    with torch.cuda.stream(stream2):
        train_and_val(model, optimizer, loss_function, 10, "./results/shufflenet_v2_x1_0.txt", "./shufflenet_v2_x1_0.pth")
    end_time = time.time()
    print(f"Training time for Model 2: {end_time - start_time:.2f} seconds")

thread1 = threading.Thread(target=train_model1)
thread2 = threading.Thread(target=train_model2)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

# Synchronize streams
stream1.synchronize()
stream2.synchronize()

Model is running on: cuda
Epoch 1:
Model is running on: cuda
Epoch 1:
Epoch [1] | Training Batch [1/451] | Loss: 0.6947
Epoch [1] | Training Batch [1/451] | Loss: 0.6951
Epoch [1] | Training Batch [26/451] | Loss: 0.6917
Epoch [1] | Training Batch [26/451] | Loss: 0.6943
Epoch [1] | Training Batch [51/451] | Loss: 0.6917
Epoch [1] | Training Batch [51/451] | Loss: 0.6904
Epoch [1] | Training Batch [76/451] | Loss: 0.6907
Epoch [1] | Training Batch [76/451] | Loss: 0.6915
Epoch [1] | Training Batch [101/451] | Loss: 0.6912
Epoch [1] | Training Batch [101/451] | Loss: 0.6903
Epoch [1] | Training Batch [126/451] | Loss: 0.6879
Epoch [1] | Training Batch [126/451] | Loss: 0.6892
Epoch [1] | Training Batch [151/451] | Loss: 0.6889
Epoch [1] | Training Batch [151/451] | Loss: 0.6891
Epoch [1] | Training Batch [176/451] | Loss: 0.6879
Epoch [1] | Training Batch [176/451] | Loss: 0.6886
Epoch [1] | Training Batch [201/451] | Loss: 0.6862
Epoch [1] | Training Batch [201/451] | Loss: 0.6861
Epoc

In [3]:
import pyvww
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

training_set = pyvww.pytorch.VisualWakeWordsClassification(
    root="./visualwakewords/path-to-COCO-dataset/train2014",
    annFile="./visualwakewords/new-path-to-visualwakewords-dataset/annotations/instances_train.json",
    transform=transform
)

validation_set = pyvww.pytorch.VisualWakeWordsClassification(
    root="./visualwakewords/path-to-COCO-dataset/train2014",
    annFile="./visualwakewords/new-path-to-visualwakewords-dataset/annotations/instances_val.json",
    transform=transform
)



training_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

loading annotations into memory...
Done (t=3.24s)
creating index...
index created!
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [ ]:
torch.cuda.empty_cache()

model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.DEFAULT).cuda()

loss_function = nn.CrossEntropyLoss()

for param in model.parameters():
    param.requires_grad = False

model.classifier[1] = torch.nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))

for param in model.classifier.parameters():
    param.requires_grad = True

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

start_time = time.time()
train_and_val(model, optimizer, loss_function, 10, "./results/squeezenet1_1.txt", "./squeezenet1_1.pth")
end_time = time.time()

print(f"Training time for Model 1: {end_time - start_time:.2f} seconds")